In [ ]:
def plot_balance_graph_with_labels(df, athlete_id, pdf, verbose=False):
    """
    Generate a balance graph for an athlete, grouped by competition level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        pdf (PdfPages): PDF object to save the plot.
        verbose (bool): Enable verbose logging for debug purposes.
    """
    # Get athlete's information
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
    athlete_level = athlete_data['Level'].iloc[-1]

    # Calculate athlete-specific values
    athlete_grip_balance = calculate_grip_balance(df, athlete_id)
    athlete_shoulder_balance = calculate_shoulder_balance(df, athlete_id)
    athlete_lateral_jump_balance = calculate_jump_balance(df, athlete_id, "Lateral")
    athlete_vertical_jump_balance = calculate_jump_balance(df, athlete_id, "Vertical")

    # Calculate facility averages for the same level
    facility_grip_avg = calculate_facility_grip_balance(df, athlete_level)
    facility_shoulder_avg = calculate_facility_shoulder_balance(df, athlete_level)
    facility_lateral_jump_avg = calculate_facility_jump_balance(df, "Lateral", athlete_level)
    facility_vertical_jump_avg = calculate_facility_jump_balance(df, "Vertical", athlete_level)

    # Define ranges and categories
    x_range = (0.5, 1.5)
    regions = {
        "Warning": [(0.5, 0.69), (1.31, 1.5)],
        "Caution": [(0.7, 0.84), (1.16, 1.3)],
        "Good": [(0.85, 0.99), (1.01, 1.15)],
        "Optimal": [(1.0, 1.0)]
    }

    # Define metrics and their values
    metrics = {
        "Shoulder Balance": {"value": athlete_shoulder_balance, "facility_avg": facility_shoulder_avg},
        "Grip Balance": {"value": athlete_grip_balance, "facility_avg": facility_grip_avg},
        "Lateral Jump Balance": {"value": athlete_lateral_jump_balance, "facility_avg": facility_lateral_jump_avg},
        "Vertical Jump Balance": {"value": athlete_vertical_jump_balance, "facility_avg": facility_vertical_jump_avg},
    }

    # Function to determine color based on balance value
    def get_color(value):
        if value is None:
            return "gray"
        if 0.5 <= value <= 0.69 or 1.31 <= value <= 1.5:
            return "red"  # Warning
        if 0.7 <= value <= 0.84 or 1.16 <= value <= 1.3:
            return "yellow"  # Caution
        if 0.85 <= value <= 0.949 or 1.051 <= value <= 1.15:
            return "green"  # Good
        if 0.95 <= value <= 1.05:
            return "blue"  # Optimal
        return "gray"

    # Set up the plot
    fig, ax = plt.subplots(figsize=(14, 5))

    # Add gradient background
    gradient = np.linspace(0.5, 1.5, 1000).reshape(1, -1)
    gradient_colors = np.abs(gradient - 1)
    ax.imshow(
        gradient_colors,
        aspect="auto",
        extent=[0.5, 1.5, 0.46, 0.54],
        cmap="coolwarm_r",
        alpha=0.8
    )

    # Draw labels for balance regions
    for region, ranges in regions.items():
        for start, end in ranges:
            ax.text(
                (start + end) / 2, 0.5,
                region,
                ha="center",
                va="center",
                fontsize=10 if region == "Optimal" else 14,
                alpha=0.2,
                fontweight="bold",
                color="black",
                transform=ax.transData
            )

    # Plot data points for each metric
    y_positions = np.linspace(0.48, 0.52, len(metrics) * 2)
    label_offsets = np.linspace(0.55, 0.59, len(metrics))
    label_h_positions = np.linspace(0.6, 1.4, len(metrics))

    for i, (metric, details) in enumerate(metrics.items()):
        value = details["value"]
        facility_avg = details["facility_avg"]

        if value is None:
            if verbose:
                print(f"Skipping {metric}: Athlete data missing.")
            continue

        color = get_color(value)
        athlete_y = y_positions[i * 2]
        label_y = label_offsets[i % len(label_offsets)] - 0.03
        label_x = label_h_positions[i % len(label_h_positions)]

        # Plot athlete data
        ax.plot(
            [value],
            [athlete_y],
            marker='o',
            color=color,
            markersize=10
        )

        # Plot facility average if available
        if facility_avg is not None:
            ax.plot(
                [facility_avg],
                [athlete_y],
                marker='x',
                color="lightgray",
                markersize=8,
                alpha=0.6
            )
            ax.plot(
                [value, facility_avg],
                [athlete_y, athlete_y],
                linestyle="dotted",
                color="gray",
                linewidth=1.5
            )

        # Add a line connecting the annotation to the data point
        ax.plot(
            [value, label_x],
            [athlete_y, label_y],
            linestyle="solid",
            color="lightblue",
            linewidth=1.5
        )

        # Annotate with metric name and balance status
        if metric == "Shoulder Balance":
            dominance = (
        "ER Dominant" if value > 1.15 else
        "IR Dominant" if value < 0.85 else
        "Balanced"
    )
        elif metric == "Grip Balance":
            dominance = (
        "Arm Side Dominant" if value > 1.05 else
        "Glove Side Dominant" if value < 0.95 else
        "Balanced"
    )
        elif metric == "Lateral Jump Balance":
            dominance = (
        "Block Leg Dominant" if value > 1.05 else
        "Load Leg Dominant" if value < 0.95 else
        "Balanced"
    )
        elif metric == "Vertical Jump Balance":
            dominance = (
        "Block Leg Dominant" if value > 1.05 else
        "Load Leg Dominant" if value < 0.95 else
        "Balanced"
    )
        else:
            dominance = "Balanced"  # Default fallback

        ax.annotate(
            f"{metric}\n{value:.2f}\n{dominance}",
            xy=(label_x, label_y),
            ha="center",
            fontsize=9,
            color="black",
            bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="white", alpha=0.8),
        )

    # Finalize plot aesthetics
    ax.set_xlim(x_range)
    ax.set_ylim(0.4, 0.6)
    ax.set_xticks(np.linspace(0.5, 1.5, 11))
    ax.set_xticklabels([f"{tick:.2f}" for tick in np.linspace(0.5, 1.5, 11)], fontsize=10)
    ax.set_yticks([])
    ax.set_xlabel("Balance Ratio", fontsize=12, fontweight="bold")
    ax.set_title(f"{athlete_name} - Imbalance Visualization\nLevel: {athlete_level}", fontsize=14, fontweight="bold")
    ax.grid(axis="x", linestyle="--", linewidth=0.5, alpha=0.7)

    # Save the plot to the PDF
    pdf.savefig(fig)
    plt.close(fig)
